# Final Project 

## Instructions

Your final project is to create a public visualization (static or interactive) using data relevant to a current policy, business, or justice issue. You may use any dataset you can find for this assignment, as long as it is either public or you have permission from the data’s owner/administrator to work with it and share it. Recommended data sources are: governmental data, data provided by a non-profit/Nongovernmental organizations, and data available from large, semi-structured data sets (ie social networks, company financials, etc). You must document eachstep of your data analysis process (excluding data acquisition) in code: this will include changing the format of the data and the creation of any images or interactive displays that are made. You must also include a short (2-3 paragraph) write-up on the visualization. This write-up must include the following: the data source, what the parameters of the data set are (geography, timeframe, what the data points are, etc) what the data shows, and why it is Important... Note - The type of deliverable you provide will depend on the strategy you use for this project. If you put together an interactive visualization, you should be able to provide code that I will be able to run and host locally. If you are choosing static visualizations, your write up will be more important to your overall grade, and it may be useful to think about how you’re presenting these visualizations (in a formated `R` Markdown document for example.)

## Proposal 

The IRS has a dataset that contains U.S. Population Migration Data from 1990 to 2016. The data “based on year-to-year address changes reported on individual income tax returns filed with the IRS.” The data shows inflows and outflows to every county in the United States as defined by US Census Bureau FIPS Codes. The flows are in terms of number of returns, number of exemptions, and aggregate income. These data are useful in business for gauging demand and human capital. The project proposed is an interactive visualization where a year, destination State, and destination County are selected, then a map with inflows or outflows by origin State and Origin County is displayed. An example would be selecting 2015 and Kings County in New York State to see a heatmap of the United States identifying from where people migrated by return count. Below are the top 15 rows of the raw data for such a selection.

|DS|DC|OS|OC|State Name|County Name|Returns|Exemptions|AGI|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|36|47|36|61|NY|New York County|12253|18042|803554000|
|36|47|36|81|NY|Queens County|8881|16332|305235000|
|36|47|36|5|NY|Bronx County|2783|5215|74961000|
|36|47|36|85|NY|Richmond County|1527|2809|64619000|
|36|47|36|59|NY|Nassau County|1317|2108|60445000|
|36|47|36|103|NY|Suffolk County|974|1502|42653000|
|36|47|34|17|NJ|Hudson County|784|1150|39661000|
|36|47|6|37|CA|Los Angeles County|770|956|31287000|
|36|47|36|119|NY|Westchester County|677|982|32442000|
|36|47|42|101|PA|Philadelphia County|568|821|20239000|
|36|47|17|31|IL|Cook County|535|668|34080000|
|36|47|6|75|CA|San Francisco County|484|557|25770000|
|36|47|34|13|NJ|Essex County|454|757|16920000|
|36|47|12|86|FL|Miami Dade County|436|648|11926000|
|36|47|34|3|NJ|Bergen County|403|605|17831000|

I plan on using `R` with `Shiny` unless there is too much data for `R` to handle. In that case, I will try to use `Python` with `plotly`. The data for this analysis are available in `CSV` Format here: [IRS U.S. Population Migration Data](https://www.irs.gov/statistics/soi-tax-stats-migration-data), [US Census Bureau FIPS Codes](https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html).

## Feedback

This is a very interesting dataset, but you still need to flesh out exactly what you're looking to show. For assignment 6, think about what kind of app you want to make (exploratory? purpose driven?), and if it is more narrative or purpose driven, think about what you want your thesis to be (highlighting the characteristics of the largest in / outflows? saying something general about how Americans move?)

## Response

I am looking to make an exploratory app that shows how Americans move in general. I was planning to do it the whole country but the millions of rows of data will make the visualization unresponsive. I have culled it down to inflows and outflows from NY which is only about a quarter-million rows of data and it still lags a little (depending on RAM), but it is functional. The app will be more purpose driven than narrative based so I plan to focus more on improving the visualization than telling a story about the data. I was a little disappointed to see that county level choropleth maps do not have zoom capabilities, but will add sliders for years and drop downs for both direction (inflow, outflow) and scope (NY to/from where). 

# U.S. Population Migration Data from 1990 to 2016

In [1]:
%%script false # DO NOT RUN CODE IN CELL
%%bash

# Brace Expansion Cartesian Product
# wget https://www.irs.gov/pub/irs-soi/{1990..2003}to{1991..2004}countymigration.zip
# wget https://www.irs.gov/pub/irs-soi/county{04..10}{05..11}.zip
# wget https://www.irs.gov/pub/irs-soi/countyinflow{11..15}{12..16}.csv
# wget https://www.irs.gov/pub/irs-soi/countyoutflow{11..15}{12..16}.csv

# FOR LOOP USED BECAUSE A {a..b}to{b..c} BRACE EXPANSION RETURNS CARTESIAN PRODUCT
for i in {1990..2003};do wget https://www.irs.gov/pub/irs-soi/${i}to$((i+1))countymigration.zip;done

# NUMBER ZERO-PADDED IN LOOP INSTEAD OF USING {04..11} BECAUSE NUMBERS WITH 
# LEADING 0 ARE BASE 8 OCTAL NUMBERS WHICH RETURN A "TOO GREAT FOR BASE" ERROR.
for i in {4..10};do wget https://www.irs.gov/pub/irs-soi/county$(printf "%02d" ${i})$(printf "%02d" $((i+1))).zip;done
for i in {11..15};do wget https://www.irs.gov/pub/irs-soi/countyinflow${i}$((i+1)).csv;done
for i in {11..15};do wget https://www.irs.gov/pub/irs-soi/countyoutflow${i}$((i+1)).csv;done

# UNZIP ALL AND REMOVE ZIP FILE
unzip "*.zip"
sudo rm *.zip

# FLATTEN AND REMOVE SUBDIRECTORIES
sudo find . -mindepth 2 -type f -exec mv -i '{}' . ';'
rm -Rf -- */

# SIZE OF DATA DOWNLOADED
du -sh .

# FORMAT OF DOWNLOADED FILES
find . -type f | awk -F. '!a[$NF]++{print $NF}'
find . -type f -name "*.dat"
find . -type f -name "*.doc"

# REMOVE ALL FILES
# rm -rf ./*

# REMOVE NON-CSV and NON-XLS
rm *.!(csv,xls)

In [2]:
import numpy as np
import pandas as pd

In [3]:
github = 'https://raw.githubusercontent.com/jzuniga123/SPS/master/DATA%20608/'

## Federal Information Processing Standards (FIPS) County Codes

In [4]:
df_fips = pd.read_csv(github + 'FIPS_Codes_2016.csv', encoding ='latin1')
df_fips.head()

,State_Code,County_Code,State,Abbreviation,County
0,0,0,United States,US,United States
1,1,0,Alabama,AL,Alabama
2,1,1,Alabama,AL,Autauga County
3,1,3,Alabama,AL,Baldwin County
4,1,5,Alabama,AL,Barbour County


## IRS Statistics of Income Division (SOI) Tax Stats - Migration Data

In [7]:
col_right = ['State_Name', 'County_Name', 'Returns', 'Exemptions', 'Income']
col_fips, df_list = (['State_Code', 'County_Code'], [])
for year, io in [(x, y) for x in range(1990, 2016) for y in ['i','o']]:
    file = github + 'IRS_NYc' + str(year) + 'to' + str(year + 1) + io
    col_basis = [('I_' if io == 'i' else 'O_') + c for c in col_fips]
    col_delta = [('I_' if io == 'o' else 'O_') + c for c in col_fips]
    cols = col_basis + col_delta + col_right
    if year <= 1991:
        df_temp = pd.read_csv(file + '.csv', skiprows=1, names=cols)
    else:
        df_temp = pd.ExcelFile(file + '.xls').parse(skiprows=(7 if year<=2008 else 5), names=cols)
    df_temp.drop(['State_Name', 'County_Name'], axis=1, inplace=True)
    df_temp['Direction'] = 'Inflow' if (io == 'i') else 'Outflow'
    df_temp['Year_From'], df_temp['Year_To'] = (year, year+1)
    for col_irs in [col_basis, col_delta]:
        df_temp = pd.merge(df_fips, df_temp, left_on=col_fips, right_on=col_irs, how='inner')
        dic = dict(zip(df_fips.columns[2:], [col_irs[0][:2] + l for l in ['State', 'Abbr', 'County']]))
        df_temp.rename(columns=dic, inplace=True)
        fips_left = df_temp[col_irs[0]].astype(int).astype(str).str.zfill(2)
        fips_right = df_temp[col_irs[1]].astype(int).astype(str).str.zfill(3)
        df_temp[col_irs[0][:2] + 'FIPS'] = fips_left + fips_right
        df_temp.drop(col_fips+col_irs, axis=1, inplace=True)
    df_list.append(df_temp)
df_irs = pd.concat(df_list, ignore_index=False).iloc[:,[12,13,0,4,2,5,3,9,7,10,8,11,1,6]]
df_irs.to_csv('IRS_NYc1990to2016io.csv', index=False)
df_irs.head()

,Year_From,Year_To,Direction,I_FIPS,I_Abbr,I_State,I_County,O_FIPS,O_Abbr,O_State,O_County,Returns,Exemptions,Income
0,1990,1991,Inflow,36065,NY,New York,Oneida County,01045,AL,Alabama,Dale County,10,19,NaN
1,1990,1991,Inflow,36047,NY,New York,Kings County,01073,AL,Alabama,Jefferson County,11,19,NaN
2,1990,1991,Inflow,36061,NY,New York,New York County,01073,AL,Alabama,Jefferson County,17,20,NaN
3,1990,1991,Inflow,36061,NY,New York,New York County,01097,AL,Alabama,Mobile County,13,25,NaN
4,1990,1991,Inflow,36045,NY,New York,Jefferson County,02020,AK,Alaska,Anchorage Municipality,14,40,NaN


# USA County Choropleth Maps

In [8]:
%%script false # DO NOT RUN CODE IN CELL
%%bash
# NEEDED FOR MPAS
sudo apt-get update
pip install --upgrade pip
pip install geopandas
pip install pyshp
pip install shapely

In [9]:
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.figure_factory as ff
offline.init_notebook_mode(connected=True)

In [10]:
# BASIS
year = 2015
direction = 'Inflow'
basis = "Kings County"
# DELTA
scope = 'USA' # State abbreviation or USA
pd.DataFrame({'Year':year,'Direction':direction, 'From':basis, 'To':scope}, index=[0])

,Direction,From,To,Year
0,Inflow,Kings County,USA,2015


In [11]:
basis_col = direction[0:1] + '_County'
delta_col = 'O_Abbr' if (direction[0:1] == "I") else 'I_Abbr'
delta_fips = 'O_FIPS' if (direction[0:1] == "I") else 'I_FIPS'
delta = scope if (scope in df_irs[delta_col].values) else df_irs[delta_col]
index = (df_irs.Year_From == year) & \
        (df_irs.Direction == direction) & \
        (df_irs[basis_col] == basis) & \
        (df_irs[delta_col] == delta)
fips = df_irs[index][delta_fips]
values = df_irs[index].Returns.values
colorscale = ["#b3d2e9","#9ecae1","#85bcdb","#6baed6","#57a0ce","#4292c6",
              "#3082be","#2171b5","#1361a9","#08519c","#0b4083","#08306b"]
fig = ff.create_choropleth(fips=fips, values=values, scope=[scope],
                           binning_endpoints=np.percentile(values,range(0,100,10)).tolist(),
                           state_outline={'color':'black', 'width': 1},
                           county_outline={'color': 'grey', 'width': 1},
                           show_hover=True,
                           show_state_data=True, # state oulines
                           round_legend_values=True,
                           dragmode = 'zoom',
                           colorscale = colorscale, 
                           legend_title = direction + 's to ' + scope,
                           title='IRS Migration Data: NY (' + basis + ") " + str(year)
                          )
offline.iplot(fig) 

In [12]:
# BASIS
year = 2015
direction = 'Inflow'
basis = "Kings County"
# DELTA
scope = 'NY' # State abbreviation or USA
pd.DataFrame({'Year':year,'Direction':direction, 'From':basis, 'To':scope}, index=[0])

,Direction,From,To,Year
0,Inflow,Kings County,NY,2015


In [13]:
basis_col = direction[0:1] + '_County'
delta_col = 'O_Abbr' if (direction[0:1] == "I") else 'I_Abbr'
delta_fips = 'O_FIPS' if (direction[0:1] == "I") else 'I_FIPS'
delta = scope if (scope in df_irs[delta_col].values) else df_irs[delta_col]
index = (df_irs.Year_From == year) & \
        (df_irs.Direction == direction) & \
        (df_irs[basis_col] == basis) & \
        (df_irs[delta_col] == delta)
fips = df_irs[index][delta_fips]
values = df_irs[index].Returns.values
colorscale = ["#b3d2e9","#9ecae1","#85bcdb","#6baed6","#57a0ce","#4292c6",
              "#3082be","#2171b5","#1361a9","#08519c","#0b4083","#08306b"]
fig = ff.create_choropleth(fips=fips, values=values, scope=[scope],
                           binning_endpoints=np.percentile(values,range(0,100,10)).tolist(),
                           state_outline={'color':'black', 'width': 1},
                           county_outline={'color': 'grey', 'width': 1},
                           show_hover=True,
                           show_state_data=True, # state oulines
                           round_legend_values=True,
                           dragmode = 'zoom',
                           colorscale = colorscale, 
                           legend_title = direction + 's to ' + scope,
                           title='IRS Migration Data: NY (' + basis + ") " + str(year)
                          )
offline.iplot(fig) 

In [14]:
from IPython.display import HTML
HTML('''
    <script>
    code_show=true; 
        function code_toggle() {
             if (code_show){
                 $('div.input').hide();
             } else {
                 $('div.input').show();
             }
             code_show = !code_show
        } 
    $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# References

https://plot.ly/python/choropleth-maps/

https://plot.ly/python/county-choropleth/

https://plot.ly/python/reference/#choropleth

https://github.com/juliangruber/brace-expansion

http://wiki.bash-hackers.org/syntax/expansion/brace

https://www.irs.gov/statistics/soi-tax-stats-migration-data

https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html

https://community.plot.ly/t/what-colorscales-are-available-in-plotly-and-which-are-the-default/2079

https://unix.stackexchange.com/questions/89113/how-to-do-multiple-brace-expansions-without-combinatorial-explosion

https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer